<a href="https://colab.research.google.com/github/nakamura196/000_tools/blob/main/ndlocr_v2_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="orange">NDLOCR（ver.2.1）の実行例（シンプル版）</font>

本ノートブックのライセンス： <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />

<br/>

NDL OCRの説明: https://github.com/ndl-lab/ndlocr_cli

<br/>


## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCRv2_googlecolabversion.ipynb

## 1:初期セットアップ

少し時間がかかります。初回のみ実行が必要です。

In [ ]:
#@title セットアップ

import yaml
from google.colab import drive
from IPython.display import clear_output

CONTENT_DIR = "/content"

# Google Drive関連
drive.mount(f'{CONTENT_DIR}/drive/')

###

!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2 --index-url https://download.pytorch.org/whl/cu118

%cd {CONTENT_DIR}

# !nvidia-smi
# !pip list | grep torch
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli -b feature/colab
# #2. 必要なパッケージをインストールする
PROJECT_DIR=f"{CONTENT_DIR}/ndlocr_cli"
# !pip install -r {PROJECT_DIR}/requirements.txt # コメントアウト

!pip install hydra-colorlog
!pip install hydra-core
!pip install mmcls==1.0.0rc6
!pip install pytorch-lightning==1.6.5
!pip install datasets

!pip install mmcv==2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

!pip install torchmetrics==0.11.4

%cd {PROJECT_DIR}/submodules/ndl_layout
!git clone https://github.com/open-mmlab/mmdetection.git -b v3.0.0
%cd {PROJECT_DIR}/submodules/ndl_layout/mmdetection
#下行はGPUのメモリ不足になった場合にコメントアウトを外すとよい。
!sed -i -e 's/GPU_MEM_LIMIT = 1024\*\*3/GPU_MEM_LIMIT = 1024\*\*3\/\/5/' mmdet/models/roi_heads/mask_heads/fcn_mask_head.py
!python setup.py bdist_wheel
!pip install dist/*.whl

# # 4. OCRに必要な学習済みモデルをダウンロードする
%cd {PROJECT_DIR}
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/resnet-orient2.ckpt -P ./submodules/text_recognition_lightning/models
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/rf_author/model.pkl -P ./submodules/text_recognition_lightning/models/rf_author/
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/text_recognition_lightning/rf_title/model.pkl -P ./submodules/text_recognition_lightning/models/rf_title/
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/ndl_layout/ndl_retrainmodel.pth -P ./submodules/ndl_layout/models
!wget -nc https://lab.ndl.go.jp/dataset/ndlocr_v2/separate_pages_mmdet/epoch_180.pth -P ./submodules/separate_pages_mmdet/models

def disable_ruby_reading_in_config():
  # 設定ファイル内でルビ読みを無効にします。
  config_path = f'{PROJECT_DIR}/config.yml'

  with open(config_path) as file:
      config = yaml.safe_load(file)
      config["ruby_read"] = False

      with open(config_path, 'w') as file:
        yaml.dump(config, file)

disable_ruby_reading_in_config()

clear_output()

## 2:OCRのみ

`xml`と`img`を含むフォルダを用意し、`s`引数に`i`を与えます。

https://github.com/ndl-lab/ndlocr_cli?tab=readme-ov-file#%E5%85%A5%E5%8A%9B%E3%83%87%E3%82%A3%E3%83%AC%E3%82%AF%E3%83%88%E3%83%AA%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6

In [ ]:
# Googleドライブ上のzipファイルを、ディスク上に解凍する
!unzip /content/drive/MyDrive/ndlocr_v2/input/head.zip -d /content/

In [ ]:
%cd /content/ndlocr_cli/

In [ ]:
# OCRの実行
!python main.py infer /content/head /content/output/head -s i -p 3

In [ ]:
# 出力結果をディスク上に圧縮する
!zip -r ../head.zip ../output

In [ ]:
# 圧縮したファイルをGoogleドライブにコピーする
!cp /content/head.zip /content/drive/MyDrive/ndlocr_v2/output/

## 3:レイアウトとOCR

`img`を含むフォルダを用意し、`s`引数に`s`を与えます。

https://github.com/ndl-lab/ndlocr_cli?tab=readme-ov-file#%E5%85%A5%E5%8A%9B%E3%83%87%E3%82%A3%E3%83%AC%E3%82%AF%E3%83%88%E3%83%AA%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6

In [ ]:
# Googleドライブ上のzipファイルを、ディスク上に解凍する
!unzip /content/drive/MyDrive/ndlocr_v2/input/text.zip -d /content/

In [ ]:
%cd /content/ndlocr_cli/

In [ ]:
# OCRの実行
!python main.py infer /content/text /content/output/text -s s -p 2..3